In [112]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [113]:
df = pd.read_csv("raw_data_com_labels.csv", index_col=0)

#### Vamos usar uma cópia do DataFrame para fazer a limpeza

In [114]:
df_limpo = df.copy()

In [115]:
df_limpo.head()

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,['75'],['2010'],[LIVRO],['3'],algoritmos+geneticos,NaN
1,https://www.researchgate.net/profile/Pablo_Est...,Optimización mediante algoritmos genéticos,['51'],['1997'],[PDF],['2'],algoritmos+geneticos,NaN
2,http://it.uc3m.es/jvillena/irc/practicas/06-07...,Algoritmos genéticos,['18'],"['2007', '3']",[PDF],['5'],algoritmos+geneticos,NaN
3,http://cursos.itam.mx/akuri/PUBLICA.CNS/2000/A...,Algoritmos genéticos,['26'],['2002'],[LIVRO],['2'],algoritmos+geneticos,NaN
4,https://the-geek.org/docs/algen/algen.html,Algoritmos genéticos y computación evolutiva,['29'],['2004'],[HTML],['2'],algoritmos+geneticos,NaN


In [116]:
##Pegamos os valores que foram anotados
df_limpo = df_limpo[df_limpo['y'].notnull()]
df_limpo.head(100)

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
6,https://books.google.com.br/books?hl=pt-BR&lr=...,Algoritmos genéticos (2a ediçao),['287'],['2008'],[LIVRO],['9'],algoritmos+geneticos,1.0
7,http://www.biblioteca.uma.es/bbldoc/tesisuma/1...,Análisis y diseño de algoritmos genéticos para...,['26'],['1999'],[PDF],['8'],algoritmos+geneticos,0.0
8,https://www.colibri.udelar.edu.uy/jspui/handle...,Algoritmos genéticos paralelos y su aplicación...,['18'],['2004'],[PDF],['5'],algoritmos+geneticos,0.0
9,https://www.researchgate.net/profile/Marcos_Ge...,Introducción a los algoritmos genéticos,['19'],['2000'],[PDF],['11'],algoritmos+geneticos,0.0
10,https://www.redalyc.org/pdf/816/81690104.pdf,Sistemas recomendadores: Un enfoque desde los ...,['22'],['2006'],[PDF],['2'],algoritmos+geneticos,0.0
...,...,...,...,...,...,...,...,...
105,https://dialnet.unirioja.es/servlet/articulo?c...,Algoritmo genético modificado chu-beasley apli...,['8'],['2007'],[CITAÇÃO],['8'],algoritmos+geneticos,0.0
111,http://www.scielo.br/scielo.php?pid=S0100-6762...,Metaheurística algoritmo genético para solução...,['6'],['2000'],[CITAÇÃO],['23'],algoritmos+geneticos,0.0
112,http://revistas.utp.edu.co/index.php/revistaci...,Algoritmo genético especializado para la local...,['7'],['2004'],[CITAÇÃO],['2'],algoritmos+geneticos,0.0
140,https://users.dcc.uchile.cl/~nbaloian/DSS-DCC/...,Generación de horarios académicos en INACAP ut...,['5'],['2014'],[DOC],['2'],algoritmos+geneticos,0.0


#### Vamos resetar os índices

In [117]:
df_limpo = df_limpo.reset_index(drop=True)
df_limpo.head(100)

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,https://books.google.com.br/books?hl=pt-BR&lr=...,Algoritmos genéticos (2a ediçao),['287'],['2008'],[LIVRO],['9'],algoritmos+geneticos,1.0
1,http://www.biblioteca.uma.es/bbldoc/tesisuma/1...,Análisis y diseño de algoritmos genéticos para...,['26'],['1999'],[PDF],['8'],algoritmos+geneticos,0.0
2,https://www.colibri.udelar.edu.uy/jspui/handle...,Algoritmos genéticos paralelos y su aplicación...,['18'],['2004'],[PDF],['5'],algoritmos+geneticos,0.0
3,https://www.researchgate.net/profile/Marcos_Ge...,Introducción a los algoritmos genéticos,['19'],['2000'],[PDF],['11'],algoritmos+geneticos,0.0
4,https://www.redalyc.org/pdf/816/81690104.pdf,Sistemas recomendadores: Un enfoque desde los ...,['22'],['2006'],[PDF],['2'],algoritmos+geneticos,0.0
...,...,...,...,...,...,...,...,...
95,https://dialnet.unirioja.es/servlet/articulo?c...,Algoritmo genético modificado chu-beasley apli...,['8'],['2007'],[CITAÇÃO],['8'],algoritmos+geneticos,0.0
96,http://www.scielo.br/scielo.php?pid=S0100-6762...,Metaheurística algoritmo genético para solução...,['6'],['2000'],[CITAÇÃO],['23'],algoritmos+geneticos,0.0
97,http://revistas.utp.edu.co/index.php/revistaci...,Algoritmo genético especializado para la local...,['7'],['2004'],[CITAÇÃO],['2'],algoritmos+geneticos,0.0
98,https://users.dcc.uchile.cl/~nbaloian/DSS-DCC/...,Generación de horarios académicos en INACAP ut...,['5'],['2014'],[DOC],['2'],algoritmos+geneticos,0.0


# Limpeza de dados

In [118]:
df_limpo.dtypes

link             object
titulo           object
citacoes         object
ano              object
tipo_arquivo     object
versao           object
busca            object
y               float64
dtype: object

### Removendo alguns caracteres descenessários nos registros

In [119]:
def clean_data(cols_list, char_list):
    for col in cols_list:
        for char in char_list:
            df_limpo[col] = df_limpo[col].str.replace(char, '')

In [120]:
cols_to_clean = ['citacoes', 'ano', 'tipo_arquivo', 'versao']
spec_chars = ["[", "]", "'", " "]

clean_data(cols_to_clean, spec_chars)
df_limpo.head()

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,https://books.google.com.br/books?hl=pt-BR&lr=...,Algoritmos genéticos (2a ediçao),287,2008,LIVRO,9,algoritmos+geneticos,1.0
1,http://www.biblioteca.uma.es/bbldoc/tesisuma/1...,Análisis y diseño de algoritmos genéticos para...,26,1999,PDF,8,algoritmos+geneticos,0.0
2,https://www.colibri.udelar.edu.uy/jspui/handle...,Algoritmos genéticos paralelos y su aplicación...,18,2004,PDF,5,algoritmos+geneticos,0.0
3,https://www.researchgate.net/profile/Marcos_Ge...,Introducción a los algoritmos genéticos,19,2000,PDF,11,algoritmos+geneticos,0.0
4,https://www.redalyc.org/pdf/816/81690104.pdf,Sistemas recomendadores: Un enfoque desde los ...,22,2006,PDF,2,algoritmos+geneticos,0.0


### Alterando o tipo das colunas

In [121]:
df_limpo[cols_to_clean] = df_limpo[cols_to_clean].astype('int')

ValueError: invalid literal for int() with base 10: '2010,200,62,146,19'

#### Existem missing values

In [122]:
df_limpo = df_limpo.dropna(how='any')

In [123]:
df_limpo.shape

(200, 8)

#### Mudamos o tipo das colunas, exceto a coluna 'ano', pois ainda precisamos limpá-la

In [124]:
cols_to_convert = ['citacoes', 'versao', 'y']
df_limpo[cols_to_convert] = df_limpo[cols_to_convert].astype('int64')

In [125]:
df_limpo.dtypes

link            object
titulo          object
citacoes         int64
ano             object
tipo_arquivo    object
versao           int64
busca           object
y                int64
dtype: object

In [126]:
for i in df_limpo['ano']:
    if(len(i)>4):
        print(i)

2010,200,62,146,19
2011,2011,40
1708,07747,2017
33,2010
2001,2001
2010,2010
2017,2017
10,939,288,2005
2011,27,2011
1307,0411,2013
1206,4656,2012
2005,2005
1702,08608,2017
21,2012
1605,07277,2016


#### É necessário retirar as vírgulas e deixar apenas os anos

In [127]:
coluna_ano = df_limpo['ano'].copy()

In [128]:
import re

for i in range(len(coluna_ano)):
    if(len(coluna_ano[i])>4):
        res = re.findall(r'\d{4}', coluna_ano[i])
        if(len(res)>1):
            coluna_ano[i] = res[len(res)-1]
        else:
            coluna_ano[i] = res[0]
print(coluna_ano)

0      2008
1      1999
2      2004
3      2000
4      2006
       ... 
195    2018
196    2017
197    2006
198    2016
199    2005
Name: ano, Length: 200, dtype: object


In [129]:
df_limpo['ano'] = coluna_ano

In [130]:
df_limpo.head()

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y
0,https://books.google.com.br/books?hl=pt-BR&lr=...,Algoritmos genéticos (2a ediçao),287,2008,LIVRO,9,algoritmos+geneticos,1
1,http://www.biblioteca.uma.es/bbldoc/tesisuma/1...,Análisis y diseño de algoritmos genéticos para...,26,1999,PDF,8,algoritmos+geneticos,0
2,https://www.colibri.udelar.edu.uy/jspui/handle...,Algoritmos genéticos paralelos y su aplicación...,18,2004,PDF,5,algoritmos+geneticos,0
3,https://www.researchgate.net/profile/Marcos_Ge...,Introducción a los algoritmos genéticos,19,2000,PDF,11,algoritmos+geneticos,0
4,https://www.redalyc.org/pdf/816/81690104.pdf,Sistemas recomendadores: Un enfoque desde los ...,22,2006,PDF,2,algoritmos+geneticos,0


In [131]:
df_limpo['ano'].astype('int32')

0      2008
1      1999
2      2004
3      2000
4      2006
       ... 
195    2018
196    2017
197    2006
198    2016
199    2005
Name: ano, Length: 200, dtype: int32

In [132]:
encoded_columns = df_limpo[['tipo_arquivo', 'busca']]
print(encoded_columns)

    tipo_arquivo                 busca
0          LIVRO  algoritmos+geneticos
1            PDF  algoritmos+geneticos
2            PDF  algoritmos+geneticos
3            PDF  algoritmos+geneticos
4            PDF  algoritmos+geneticos
..           ...                   ...
195        LIVRO      machine+learning
196        LIVRO      machine+learning
197        LIVRO      machine+learning
198        LIVRO      machine+learning
199          PDF      machine+learning

[200 rows x 2 columns]


In [133]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

enc_columns = pd.DataFrame(encoder.fit_transform(encoded_columns).toarray())
enc_columns.head()

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [134]:
new_labels = ['CITACOES', 'DOC', 'HTML', 'LIVRO', 'PDF', 'algoritmos_geneticos', 'machine_learning']
enc_columns.columns = new_labels
enc_columns

,CITACOES,DOC,HTML,LIVRO,PDF,algoritmos_geneticos,machine_learning
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
195,0.0,0.0,0.0,1.0,0.0,0.0,1.0
196,0.0,0.0,0.0,1.0,0.0,0.0,1.0
197,0.0,0.0,0.0,1.0,0.0,0.0,1.0
198,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [135]:
df_limpo = df_limpo.join(enc_columns)
df_limpo.head()

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca,y,CITACOES,DOC,HTML,LIVRO,PDF,algoritmos_geneticos,machine_learning
0,https://books.google.com.br/books?hl=pt-BR&lr=...,Algoritmos genéticos (2a ediçao),287,2008,LIVRO,9,algoritmos+geneticos,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,http://www.biblioteca.uma.es/bbldoc/tesisuma/1...,Análisis y diseño de algoritmos genéticos para...,26,1999,PDF,8,algoritmos+geneticos,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,https://www.colibri.udelar.edu.uy/jspui/handle...,Algoritmos genéticos paralelos y su aplicación...,18,2004,PDF,5,algoritmos+geneticos,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,https://www.researchgate.net/profile/Marcos_Ge...,Introducción a los algoritmos genéticos,19,2000,PDF,11,algoritmos+geneticos,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,https://www.redalyc.org/pdf/816/81690104.pdf,Sistemas recomendadores: Un enfoque desde los ...,22,2006,PDF,2,algoritmos+geneticos,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [136]:
df_limpo = df_limpo.drop(columns=['tipo_arquivo', 'busca'])
df_limpo.head()

,link,titulo,citacoes,ano,versao,y,CITACOES,DOC,HTML,LIVRO,PDF,algoritmos_geneticos,machine_learning
0,https://books.google.com.br/books?hl=pt-BR&lr=...,Algoritmos genéticos (2a ediçao),287,2008,9,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,http://www.biblioteca.uma.es/bbldoc/tesisuma/1...,Análisis y diseño de algoritmos genéticos para...,26,1999,8,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,https://www.colibri.udelar.edu.uy/jspui/handle...,Algoritmos genéticos paralelos y su aplicación...,18,2004,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,https://www.researchgate.net/profile/Marcos_Ge...,Introducción a los algoritmos genéticos,19,2000,11,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,https://www.redalyc.org/pdf/816/81690104.pdf,Sistemas recomendadores: Un enfoque desde los ...,22,2006,2,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [137]:
df_limpo.to_feather("data_cleaned.feather")